In [21]:
# !unzip /content/Test.zip

In [22]:
# Test Path 
testPath = 'Test'

# In colob testPath is 
# testPath = 'content/Test'

In [23]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import feature
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from itertools import chain

In [24]:
from utils import create_images_list
from constants import (BACKGROUND_PATH)

In [25]:
suffix = '.pgm'
kaggle_files = os.listdir(testPath)
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [26]:
len(pgm_kaggle_files)

8000

In [27]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = testPath + '/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

100%|██████████| 8000/8000 [00:01<00:00, 5572.75it/s]


In [28]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 3221740.95it/s]


In [29]:
facesPath = 'Faces' # use '/content/Faces' on colab
faces = create_images_list([facesPath])
faces.shape

100%|██████████| 12833/12833 [00:02<00:00, 6339.27it/s]


(12833, 64, 64)

In [30]:
fondos = create_images_list([BACKGROUND_PATH])
fondos.shape


100%|██████████| 12800/12800 [00:02<00:00, 4577.72it/s]


(12800, 64, 64)

In [31]:
images = list(faces) + list(fondos)

In [32]:
X = np.array([feature.hog(im) for im in tqdm(chain(faces, fondos), desc='Construyendo X')])
y = np.zeros(len(X))
faces_len = len(faces)
y[:faces_len]= 1

Construyendo X: 25633it [00:13, 1929.67it/s]


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
X_train_std = StandardScaler().fit_transform(X_train)

In [40]:
pca = PCA(n_components=500, whiten=False).fit(X_train_std)
X_train_pca = pca.transform(X_train_std)

In [41]:
# Aquí debe utilizar su modelo
model = GaussianNB()
model.fit(X_train_pca, y_train)

GaussianNB()

In [42]:
# Aquí debe procesar las imagenes de pgm_kaggle_files
X_kag = np.array([feature.hog(im) for im in tqdm(kaggle_patches, desc='Construyendo X')])
# X_kag_std = StandardScaler().transform(X_kag)
scaler = StandardScaler().fit(X_kag)
X_kag_std = scaler.transform(X_kag)
X_pca_kag = pca.transform(X_kag_std)


Construyendo X: 100%|██████████| 8000/8000 [00:03<00:00, 2020.86it/s]


In [43]:
# Predicción
y_kag = model.predict(X_pca_kag)


In [44]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [45]:
kaggle_hat.head()

,id,target_feature
2069,0,0
2527,1,0
3030,2,0
2576,3,0
4060,4,1


In [46]:
# Guardamos en un .csv para subir a kaggle
submission_name = 'submission_name'
kaggle_hat.to_csv(submission_name + '.csv', index=False)